In [1]:
import shapely
import geopandas as gpd
import pandas as pd
import numpy as np

### County Shapefiles

U.S. County shapefiles downloaded from U.S. Census at their TIGER/Line Shapefile portal [here](https://www.census.gov/cgi-bin/geo/shapefiles/index.php). We used the most recent shapefiles updated in 2019 and selected Counties (and equivalent) as our layer. A table of FIP section codes by county per 2018 can be found [here](https://www.census.gov/geographies/reference-files/2018/demo/popest/2018-fips.html).

This project focuses on the contiguous 48 states, therefore we drop entries for the states of Alaska and Hawaii as well as U.S. island territories such as American Samoa, Guam, and Peurto Rice.

In [2]:
counties = gpd.read_file('../../project_4_data/us_counties/tl_2019_us_county.shp')
counties.columns = counties.columns.str.lower()
counties.rename(columns = {'name':'county_name'}, inplace=True)

# Subset state counties
counties = counties.loc[~counties['statefp'].isin(['02','15','60','66','69','72','78']), :]

# Keep only FIP, name, and geometry
counties = counties.loc[:,counties.columns.isin(['geoid', 'state','county_name', 'geometry'])]

# Convert to coordinates system used by NOAA
counties = counties.to_crs('epsg:4326')

### COVID and Pop Data

All COVID-19 case data were pulled from John Hopkins University Center for Systems Science and Engineering's publicly accessible github repo, which is updated daily [here](https://github.com/CSSEGISandData/COVID-19). The tracking files already incorporate population demographics information from the American Commuities Survey conducted by the Census Bureau, removing the need to download additioal data files directly from the Census Bureau.



In [3]:
covid = pd.read_csv('../../project_4_data/Enriched_JHU_Centers_for_Civic_Impact_Covid_19_County_Cases.csv')

covid.columns = covid.columns.str.lower().str.replace(' ','_')
keep = ['fips_code', 'state_abbreviation',
        'county_confirmed_divided_by_county_population_*_100,000',
        'county_deaths_divided_by_county_population_*_100,000',
        '2019_population_density',
        'total_population']
covid.drop(columns = covid.columns[~covid.columns.isin(keep)], inplace=True)

covid.rename(columns = {'fips_code':'fips', 'state_abbreviation':'state',
              'county_confirmed_divided_by_county_population_*_100,000':'cc_per_100kpop',
             'county_deaths_divided_by_county_population_*_100,000':'cd_per_100kpop',
             'total_population': 'pop', 
              '2019_population_density':'pop_density'}, inplace=True)

covid['fips'] = covid['fips'].astype(str).str.zfill(5)

In [4]:
counties = counties.merge(covid, left_on='geoid', right_on='fips')

### Tornado

Our data on tornados event come from NOAA's Storm Prediction Center zipfiles for extreme weather events available [here](https://www.spc.noaa.gov/gis/svrgis/). We isolate all incidents since 2007 as these were graded on a consistent scale and with consistent data and it provides a nice clear 10 year interval basis. Documentation provided [here](https://www.spc.noaa.gov/wcm/data/SPC_severe_database_description.pdf). Reference for tornado magnitudes available [here](https://www.weather.gov/mkx/taw-tornado_classification_safety).

Below we tabulate tornado events by county using recorded geopaths.

In [143]:
tornados = gpd.read_file('../../project_4_data/1950-2018-torn-aspath/1950-2018-torn-aspath.shp')

tornados = tornados.loc[tornados['yr'] >= 2007, :]
tornados = tornados.loc[: ,['mag', 'inj', 'fat', 'loss', 'geometry']]
tornados.columns = ['magnitude', 'injuries', 'fatalities', 'econ_loss', 'geometry']

t_merge = gpd.sjoin(counties, tornados, how='left', op='intersects')

# Sum injuries and fatalities per county
injuries = t_merge.groupby(['geoid'])['injuries'].sum().reset_index()
fatalities = t_merge.groupby(['geoid'])['fatalities'].sum().reset_index()

# Tabulate tornados counts per county by magnitude
mag_events = t_merge.groupby(['geoid','magnitude']).count().reset_index()
mag_events = mag_events.iloc[:,0:3]
mag_events.columns=['geoid','magnitude', 'count']
mag_events = mag_events.pivot(index='geoid', columns='magnitude', values='count').reset_index()
mag_events.columns = ['geoid', 'tcount_unknown', 'tcount_f0', 'tcount_f1', 'tcount_f2', 'tcount_f3', 'tcount_f4', 'tcount_f5']
mag_events['tcount_all'] = mag_events.drop('geoid', axis=1).sum(axis=1)
mag_events['tcount_f3_above'] = mag_events[['tcount_f4','tcount_f4','tcount_f5']].sum(axis=1)

# Concat tornado summaries together
t_events = mag_events.merge(right=injuries, how='left', on='geoid')
t_events = t_events.merge(right=fatalities, how='left', on='geoid')
t_events.replace({np.nan:0}, inplace=True)

# Merge with counties file to tornados summary stats
torn_summary = counties.merge(right=t_events, how='left', on='geoid')

### Earthquake

To estimate risk of earthquake, we utilize the USGS the 2018 Update of the U.S. National Seismic Hazard Model. We download catalog item 3, Chance of Potentially Damaging Ground Shaking (MMI = VI) in 100 years, available [here](https://www.sciencebase.gov/catalog/item/5d55d0a5e4b01d82ce8eafa9). 

For each county, we average these probabilties across bins residing within county borders to estimate county risk. Our results closely match the non-aggregated GIS map provided.

In [46]:
earthquake = pd.read_csv('../../project_4_data/ProbMMI_VI_100Yrs_VariableVs30.csv')
earthquake = gpd.GeoDataFrame(earthquake, geometry=gpd.points_from_xy(earthquake.lon, earthquake.lat))

e_merge = gpd.sjoin(counties[['geoid','geometry']], earthquake, how='left', op='intersects')
e_merge = e_merge.groupby(['geoid']).mean().reset_index()
e_merge.rename({'PctProb100yrs':'earthquake_prob_100years'}, inplace=True)
e_merge.drop(columns=['index_right','lon','lat'], inplace=True)
earthq_summary = counties.merge(right=e_merge, how='left', on='geoid')

C:\Users\james\anaconda3\envs\geo_env2\lib\site-packages\geopandas\tools\sjoin.py:58: UserWarning: CRS of frames being joined does not match!(epsg:4326 != None)
  warn(


### Drought

Current and historical drought conditions are tracked by the National Drought Mitigation Center based at University of Nebrasks - Lincoln [here](https://droughtmonitor.unl.edu/).

In [5]:
drought = gpd.read_file('../../project_4_data/drought/dm_export_county.csv')
drought['D0'] = drought['D0'].astype(float)
drought['D1'] = drought['D1'].astype(float)
drought['D2'] = drought['D2'].astype(float)
drought['D3'] = drought['D3'].astype(float)
drought['D4'] = drought['D4'].astype(float)
drought['D_Index'] = drought['D0'] + 2*drought['D1'] + 3*drought['D2']  + 4*drought['D3']  + 5*drought['D4'] 
keep = ['FIPS', 'D0', 'D1', 'D2', 'D3', 'D4', 'D_Index']
drought.drop(columns = drought.columns[~drought.columns.isin(keep)] , inplace=True)
drought_area = counties.merge(drought, left_on = 'geoid', right_on = 'FIPS')
drought_area.drop(columns=['FIPS'], inplace=True)

### Wildfire

Raw data unavailable for wildfire risks developed by the U.S. Forest Service's Wildland Fire Assessment System, however a GIS layer is hosted publicy from an updated risk model in 2010 by U.S. Forest Service's Fire Modeling Institute, hosted [here](https://www.arcgis.com/home/item.html?id=fc0ccb504be142b59eb16a7ef44669a3).

### Hurricanes

An excellent ArcGSI storyboard for historical hurricane impacts can be found [here](https://www.arcgis.com/apps/Cascade/index.html?appid=8f6013fdba6445e9a8732ff6cab9cd1a). The data this story was based on is no longer available, but an alternative dataset of hurricane paths can be found via NOAA IBTrACS v4 data available [here](https://www.ncdc.noaa.gov/ibtracs/index.php?name=ib-v4-access) with documentation provided [here](https://www.ncdc.noaa.gov/ibtracs/pdf/IBTrACS_v04_column_documentation.pdf).

In [48]:
hurricanes = gpd.read_file('../../project_4_data/IBTrACS/IBTrACS.since1980.list.v04r00.lines.shp')
hurricanes.columns = hurricanes.columns.str.lower()

# Filter for north American storms
hurricanes = hurricanes.loc[hurricanes['basin']=='NA' , : ]

# Filter for hurricanes only
hurricanes = hurricanes.loc[hurricanes['usa_status'].isin(['HU','HR'])]

# Filter for data approaching or after landfall 
hurricanes = hurricanes.loc[hurricanes['landfall']==0 , : ]

# Filter for hurricanes greater than cat 1 
hurricanes = hurricanes.loc[hurricanes['usa_sshs'] > 0 , : ]

# Keep only path geometry, strength category, and radii variables
hurricanes = hurricanes.loc[:,['sid','usa_sshs','geometry']]

print(f"Our dataset contains")
print(f"Unique Hurricanes: {hurricanes.loc[hurricanes['usa_sshs'] > 0 , 'sid' ].drop_duplicates().shape[0]} ")
print(f"Unique Hurricanes > Cat 3: {hurricanes.loc[hurricanes['usa_sshs'] > 2 , 'sid' ].drop_duplicates().shape[0]} ")


h_merge = gpd.sjoin(counties, hurricanes, how='left', op='intersects')

# Remove duplicates and take maximum strength of hurricane in county
hurr_events = h_merge.loc[:,['geoid', 'sid', 'usa_sshs']].drop_duplicates()
hurr_events = hurr_events.groupby(['geoid', 'sid']).max().reset_index()

# Tabulate hurricanes per county by category strength 
hurr_events = hurr_events.pivot_table(index='geoid', columns='usa_sshs', values='sid', aggfunc = 'count').reset_index()
hurr_events.columns = ['geoid', 'hcount_cat1', 'hcount_cat2', 'hcount_cat3', 'hcount_cat4', 'hcount_cat5']

# Sum all strengths
hurr_events['hcount_all'] = hurr_events.drop('geoid', axis=1).sum(axis=1)
hurr_events['hcount_major'] = hurr_events[['hcount_cat3','hcount_cat4','hcount_cat5']].sum(axis=1)

hurr_summary = counties.merge(right=hurr_events, how='left', on='geoid')

Our dataset contains
Unique Hurricanes: 100 
Unique Hurricanes > Cat 3: 37 


## Cross-Tabulation Tables

In [95]:
counties['pop'] = round(counties['pop'], -3)
counties['cc_per_100kpop'] = round(counties['cc_per_100kpop'], 0).astype(int)
counties['cd_per_100kpop'] = round(counties['cd_per_100kpop'], 0).astype(int)


#### Bad Covid Areas

In [96]:
counties.loc[counties['pop_density'] > 1000,
            ['county_name','state', 'cc_per_100kpop','pop']].sort_values('cc_per_100kpop', ascending=False).head(10)

,county_name,state,cc_per_100kpop,pop
2717,Rockland,NY,3839,324000
1091,Westchester,NY,3253,969000
2558,Passaic,NJ,2924,504000
3040,Bronx,NY,2915,1438000
975,Nassau,NY,2829,1357000
1346,Richmond,NY,2674,474000
1293,Hudson,NJ,2615,669000
459,Union,NJ,2545,553000
428,Suffolk,NY,2502,1488000
2238,Queens,NY,2497,2299000


In [97]:
counties.loc[counties['pop_density'] > 1000,
            ['county_name','state', 'cd_per_100kpop','pop']].sort_values('cd_per_100kpop', ascending=False).head(10)

,county_name,state,cd_per_100kpop,pop
3040,Bronx,NY,227,1438000
2238,Queens,NY,199,2299000
2175,Essex,NJ,181,794000
2307,Kings,NY,178,2601000
2717,Rockland,NY,178,324000
459,Union,NJ,161,553000
2558,Passaic,NJ,152,504000
1346,Richmond,NY,151,474000
2277,Bergen,NJ,148,930000
1293,Hudson,NJ,148,669000


#### Bad earthquake areas

In [104]:
earthq_summary['pop'] = round(earthq_summary['pop'], -3)
earthq_summary['cc_per_100kpop'] = round(earthq_summary['cc_per_100kpop'], 0).astype(int)
earthq_summary['cd_per_100kpop'] = round(earthq_summary['cd_per_100kpop'], 0).astype(int)
earthq_summary['PctProb100yrs'] = round(earthq_summary['PctProb100yrs'], 1)

In [105]:
earthq_summary.loc[(earthq_summary['PctProb100yrs'] > 50) & (earthq_summary['pop_density'] > 1000),
            ['county_name','state', 'PctProb100yrs','cc_per_100kpop','pop']].sort_values('cc_per_100kpop', ascending=False).head(10)

,county_name,state,PctProb100yrs,cc_per_100kpop,pop
385,Los Angeles,CA,92.3,329,10098000
1937,King,WA,73.6,320,2163000
612,San Francisco,CA,85.4,224,870000
2762,San Mateo,CA,97.3,195,766000
2010,Alameda,CA,98.8,128,1644000
1862,Santa Clara,CA,99.4,122,1922000
2447,Orange,CA,95.6,113,3164000
309,Multnomah,OR,52.8,112,799000
1087,Contra Costa,CA,99.0,93,1133000
315,Sacramento,CA,80.5,77,1510000


In [106]:
earthq_summary.loc[(earthq_summary['PctProb100yrs'] > 50) & (earthq_summary['pop_density'] > 1000),
            ['county_name','state', 'PctProb100yrs','cc_per_100kpop','pop']].sort_values('PctProb100yrs', ascending=False).head(10)

,county_name,state,PctProb100yrs,cc_per_100kpop,pop
1862,Santa Clara,CA,99.4,122,1922000
1087,Contra Costa,CA,99.0,93,1133000
2010,Alameda,CA,98.8,128,1644000
2762,San Mateo,CA,97.3,195,766000
2447,Orange,CA,95.6,113,3164000
385,Los Angeles,CA,92.3,329,10098000
612,San Francisco,CA,85.4,224,870000
315,Sacramento,CA,80.5,77,1510000
1937,King,WA,73.6,320,2163000
309,Multnomah,OR,52.8,112,799000


#### Bad Hurricane Areas

In [108]:
hurr_summary['pop'] = round(hurr_summary['pop'], -3)
hurr_summary['cc_per_100kpop'] = round(hurr_summary['cc_per_100kpop'], 0).astype(int)
hurr_summary['cd_per_100kpop'] = round(hurr_summary['cd_per_100kpop'], 0).astype(int)

In [122]:
h_cols = ['county_name','state','hcount_cat1','hcount_cat2','hcount_cat3','hcount_cat4','hcount_cat5','hcount_all','hcount_major']
hurr_summary.loc[:,h_cols].sort_values('hcount_all', ascending=False).head(10)

,county_name,state,hcount_cat1,hcount_cat2,hcount_cat3,hcount_cat4,hcount_cat5,hcount_all,hcount_major
2883,Brunswick,NC,3.0,2.0,1.0,NaN,NaN,6.0,1.0
1908,New Hanover,NC,2.0,3.0,NaN,NaN,NaN,5.0,0.0
1469,Plaquemines,LA,4.0,NaN,NaN,1.0,NaN,5.0,1.0
1954,Pender,NC,1.0,3.0,1.0,NaN,NaN,5.0,1.0
1930,Polk,FL,2.0,1.0,NaN,1.0,NaN,4.0,1.0
1266,Galveston,TX,2.0,1.0,1.0,NaN,NaN,4.0,1.0
1863,Charleston,SC,3.0,NaN,NaN,1.0,NaN,4.0,1.0
675,Carteret,NC,2.0,2.0,NaN,NaN,NaN,4.0,0.0
1988,St. Lucie,FL,2.0,1.0,1.0,NaN,NaN,4.0,1.0
1344,Monroe,FL,3.0,NaN,NaN,NaN,1.0,4.0,1.0


In [124]:
h_cols = ['county_name','state','hcount_all','hcount_major','cc_per_100kpop', 'pop']
hurr_summary.loc[:,h_cols].sort_values('hcount_all', ascending=False).head(10)

,county_name,state,hcount_all,hcount_major,cc_per_100kpop,pop
2883,Brunswick,NC,6.0,1.0,37,127000
1908,New Hanover,NC,5.0,0.0,50,224000
1469,Plaquemines,LA,5.0,1.0,816,23000
1954,Pender,NC,5.0,1.0,63,59000
1930,Polk,FL,4.0,1.0,95,669000
1266,Galveston,TX,4.0,1.0,203,327000
1863,Charleston,SC,4.0,1.0,122,395000
675,Carteret,NC,4.0,0.0,50,69000
1988,St. Lucie,FL,4.0,1.0,90,306000
1344,Monroe,FL,4.0,1.0,127,76000


In [131]:
h_cols = ['county_name','state','hcount_all','hcount_major','cc_per_100kpop', 'pop']
hurr_summary.loc[( hurr_summary['hcount_all'] > 1) & (hurr_summary['pop_density'] > 500),
                 h_cols].sort_values('cc_per_100kpop', ascending=False).head(10)

,county_name,state,hcount_all,hcount_major,cc_per_100kpop,pop
1892,Jefferson,LA,3.0,0.0,1564,435000
359,Miami-Dade,FL,3.0,1.0,521,2716000
274,Broward,FL,3.0,0.0,306,1909000
302,Palm Beach,FL,2.0,0.0,275,1446000
1266,Galveston,TX,4.0,1.0,203,327000
1271,Harris,TX,3.0,0.0,179,4603000
1593,Lee,FL,2.0,1.0,175,719000
3078,Hillsborough,FL,2.0,0.0,103,1379000
1988,St. Lucie,FL,4.0,1.0,90,306000
973,Pasco,FL,2.0,0.0,56,511000


#### Bad Tornado Areas

In [139]:
torn_summary['pop'] = round(torn_summary['pop'], -3)
torn_summary['cc_per_100kpop'] = round(torn_summary['cc_per_100kpop'], 0).astype(int)
torn_summary['cd_per_100kpop'] = round(torn_summary['cd_per_100kpop'], 0).astype(int)
t_cols = ['county_name','state','tcount_f1','tcount_f2','tcount_f3','tcount_f4','tcount_f5','tcount_all']


In [150]:
tab1 = torn_summary.loc[( torn_summary['state'] == 'KS'),
                 t_cols].sort_values('tcount_all', ascending=False).head(10)
tab1['tcount_f1']=tab1['tcount_f1'].astype(int)
tab1['tcount_f2']=tab1['tcount_f2'].astype(int)
tab1['tcount_f3']=tab1['tcount_f3'].astype(int)
tab1['tcount_f4']=tab1['tcount_f4'].astype(int)
tab1['tcount_f5']=tab1['tcount_f5'].astype(int)
tab1['tcount_all']=tab1['tcount_all'].astype(int)

In [151]:
tab1

,county_name,state,tcount_f1,tcount_f2,tcount_f3,tcount_f4,tcount_f5,tcount_all
1023,Barton,KS,10,1,1,0,0,37
1329,Gove,KS,9,2,0,1,0,35
1362,Ford,KS,11,5,3,0,0,35
2484,Kiowa,KS,5,2,4,0,1,34
496,Cowley,KS,8,1,0,0,0,34
2459,Sheridan,KS,5,3,0,0,0,30
2291,Stafford,KS,10,4,4,0,0,29
246,Lane,KS,5,1,0,0,0,27
2999,Gray,KS,5,2,1,0,0,26
1137,Scott,KS,5,1,0,0,0,24


In [154]:
torn_summary.loc[torn_summary['state'] == 'KS', 
                 ['county_name', 'state','tcount_all','cc_per_100kpop', 'pop']
                ].sort_values('tcount_all', ascending=False).head(10)

,county_name,state,tcount_all,cc_per_100kpop,pop
1023,Barton,KS,37.0,88,27000
1329,Gove,KS,35.0,38,3000
1362,Ford,KS,35.0,3453,34000
2484,Kiowa,KS,34.0,79,3000
496,Cowley,KS,34.0,9,36000
2459,Sheridan,KS,30.0,79,3000
2291,Stafford,KS,29.0,24,4000
246,Lane,KS,27.0,0,2000
2999,Gray,KS,26.0,133,6000
1137,Scott,KS,24.0,82,5000


In [155]:
tab1 = torn_summary.loc[( torn_summary['state'] == 'OK'),
                 t_cols].sort_values('tcount_all', ascending=False).head(10)
tab1['tcount_f1']=tab1['tcount_f1'].astype(int)
tab1['tcount_f2']=tab1['tcount_f2'].astype(int)
tab1['tcount_f3']=tab1['tcount_f3'].astype(int)
tab1['tcount_f4']=tab1['tcount_f4'].astype(int)
tab1['tcount_f5']=tab1['tcount_f5'].astype(int)
tab1['tcount_all']=tab1['tcount_all'].astype(int)
tab1

,county_name,state,tcount_f1,tcount_f2,tcount_f3,tcount_f4,tcount_f5,tcount_all
1766,Osage,OK,12,4,0,0,0,35
595,Mayes,OK,17,2,0,0,0,27
1644,Oklahoma,OK,9,1,2,1,0,26
1687,Cleveland,OK,12,2,0,4,1,26
490,Caddo,OK,6,1,1,0,0,25
77,Canadian,OK,9,2,1,0,1,25
1537,Delaware,OK,13,6,1,0,0,22
2169,McClain,OK,10,0,0,2,1,22
2208,Beaver,OK,3,3,0,0,0,22
2172,Le Flore,OK,12,4,0,0,0,22


In [156]:
torn_summary.loc[torn_summary['state'] == 'OK', 
                 ['county_name', 'state','tcount_all','cc_per_100kpop', 'pop']
                ].sort_values('tcount_all', ascending=False).head(10)

,county_name,state,tcount_all,cc_per_100kpop,pop
1766,Osage,OK,35.0,191,47000
595,Mayes,OK,27.0,63,41000
1644,Oklahoma,OK,26.0,126,782000
1687,Cleveland,OK,26.0,162,277000
490,Caddo,OK,25.0,376,29000
77,Canadian,OK,25.0,79,137000
1537,Delaware,OK,22.0,220,42000
2169,McClain,OK,22.0,213,39000
2208,Beaver,OK,22.0,357,5000
2172,Le Flore,OK,22.0,26,50000


In [15]:
tab2 = drought_area.loc[drought_area['pop_density'] > 1000,
                 ['county_name','state','D1','D2','D3','D_Index','cc_per_100kpop', 'pop']
                ].sort_values('D_Index', ascending=False).head(10)

In [16]:
tab2.sort_values('cc_per_100kpop', ascending=False).head(10)

,county_name,state,D1,D2,D3,D_Index,cc_per_100kpop,pop
2860,Orleans,LA,93.16,0.00,0.0,286.32,1718.13,389648
1892,Jefferson,LA,100.00,0.00,0.0,300.00,1564.10,435300
359,Miami-Dade,FL,93.80,67.50,0.0,490.10,520.90,2715516
612,San Francisco,CA,100.00,100.00,0.0,600.00,223.82,870044
2762,San Mateo,CA,82.92,23.68,0.0,336.88,194.53,765935
2010,Alameda,CA,100.00,44.56,0.0,433.68,127.97,1643700
309,Multnomah,OR,74.46,51.56,0.0,403.60,111.84,798647
221,Orange,FL,99.88,0.00,0.0,299.76,111.32,1321194
1087,Contra Costa,CA,100.00,71.53,0.0,514.59,92.68,1133247
315,Sacramento,CA,100.00,81.13,0.0,543.39,76.64,1510023


In [14]:
drought_area.loc[drought_area['pop_density'] > 1000,
                 ['county_name','state','D1','D2','D3','D_Index','cc_per_100kpop','pop']
                ].sort_values('D_Index', ascending=False).head(10)

,county_name,state,D1,D2,D3,D_Index,cc_per_100kpop,pop
612,San Francisco,CA,100.00,100.00,0.0,600.00,223.82,870044
315,Sacramento,CA,100.00,81.13,0.0,543.39,76.64,1510023
1087,Contra Costa,CA,100.00,71.53,0.0,514.59,92.68,1133247
359,Miami-Dade,FL,93.80,67.50,0.0,490.10,520.90,2715516
2010,Alameda,CA,100.00,44.56,0.0,433.68,127.97,1643700
309,Multnomah,OR,74.46,51.56,0.0,403.60,111.84,798647
2762,San Mateo,CA,82.92,23.68,0.0,336.88,194.53,765935
1892,Jefferson,LA,100.00,0.00,0.0,300.00,1564.10,435300
221,Orange,FL,99.88,0.00,0.0,299.76,111.32,1321194
2860,Orleans,LA,93.16,0.00,0.0,286.32,1718.13,389648
